# Word Embeddings

### Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import pickle
import re

torch.manual_seed(1)

### Check GPU

In [2]:
gpu_status = 'OK' if torch.cuda.is_available() else 'Unavailable'
print(f'Estado GPU: {gpu_status}')
print(f'GPU: {torch.cuda.get_device_name(0)}')

Estado GPU: OK
GPU: NVIDIA GeForce RTX 3060


### Read Data

In [3]:
with open('data/MotivosDeProteo_HastaXXX.txt','r',encoding='utf-8') as f:
    text = f.read()

In [4]:
text[:600]

'Reformarse es vivir. Nuestra transformación personal en el tiempo.\n\n\nReformarse es vivir... Y desde luego, nuestra transformación personal en cierto grado ¿no es ley constante e infalible en el tiempo? ¿Qué importa que el deseo y la voluntad queden en un punto si el tiempo pasa y nos lleva? El tiempo es el sumo innovador. Su potestad, bajo la cual cabe todo lo creado, se ejerce de manera tan segura y continua sobre las almas como sobre las cosas. Cada pensamiento de tu mente, cada movimiento de tu sensibilidad, cada determinación de tu albedrío, y aun más: cada instante de la aparente tregua d'

### Tokenizer casero

In [5]:
PATTERN = r"\w+"
text_tuneado = re.findall(PATTERN, text)
text_tuneado

['Reformarse',
 'es',
 'vivir',
 'Nuestra',
 'transformación',
 'personal',
 'en',
 'el',
 'tiempo',
 'Reformarse',
 'es',
 'vivir',
 'Y',
 'desde',
 'luego',
 'nuestra',
 'transformación',
 'personal',
 'en',
 'cierto',
 'grado',
 'no',
 'es',
 'ley',
 'constante',
 'e',
 'infalible',
 'en',
 'el',
 'tiempo',
 'Qué',
 'importa',
 'que',
 'el',
 'deseo',
 'y',
 'la',
 'voluntad',
 'queden',
 'en',
 'un',
 'punto',
 'si',
 'el',
 'tiempo',
 'pasa',
 'y',
 'nos',
 'lleva',
 'El',
 'tiempo',
 'es',
 'el',
 'sumo',
 'innovador',
 'Su',
 'potestad',
 'bajo',
 'la',
 'cual',
 'cabe',
 'todo',
 'lo',
 'creado',
 'se',
 'ejerce',
 'de',
 'manera',
 'tan',
 'segura',
 'y',
 'continua',
 'sobre',
 'las',
 'almas',
 'como',
 'sobre',
 'las',
 'cosas',
 'Cada',
 'pensamiento',
 'de',
 'tu',
 'mente',
 'cada',
 'movimiento',
 'de',
 'tu',
 'sensibilidad',
 'cada',
 'determinación',
 'de',
 'tu',
 'albedrío',
 'y',
 'aun',
 'más',
 'cada',
 'instante',
 'de',
 'la',
 'aparente',
 'tregua',
 'de',
 '

### N-grama modelado simple

In [6]:
# Params de modelo
CONTEXT_SIZE = 15
EMBEDDING_DIM = 64
HIDDEN_DIM = 256
EPOCHS = 100
LEARNING_RATE = 0.001

In [7]:
ngrams = [
    (
        [text_tuneado[i - j - 1] for j in range(CONTEXT_SIZE)],
        text_tuneado[i]
    )
    for i in range(CONTEXT_SIZE, len(text_tuneado))
]

print(ngrams[:3])


[(['luego', 'desde', 'Y', 'vivir', 'es', 'Reformarse', 'tiempo', 'el', 'en', 'personal', 'transformación', 'Nuestra', 'vivir', 'es', 'Reformarse'], 'nuestra'), (['nuestra', 'luego', 'desde', 'Y', 'vivir', 'es', 'Reformarse', 'tiempo', 'el', 'en', 'personal', 'transformación', 'Nuestra', 'vivir', 'es'], 'transformación'), (['transformación', 'nuestra', 'luego', 'desde', 'Y', 'vivir', 'es', 'Reformarse', 'tiempo', 'el', 'en', 'personal', 'transformación', 'Nuestra', 'vivir'], 'personal')]


In [8]:
# encoder
vocab = set(text_tuneado)
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [9]:
# Modelo

class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size, hidden_dim):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [10]:
model = NGramLanguageModeler(
                            len(vocab),
                            EMBEDDING_DIM,
                            CONTEXT_SIZE,
                            HIDDEN_DIM).cuda()

In [11]:
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [12]:
losses = []

for epoch in range(EPOCHS):
    total_loss = 0
    for context, target in ngrams:

        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long).cuda()

        model.zero_grad()
        
        log_probs = model(context_idxs)

        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long).cuda())

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f'Epoch: {epoch} Loss: {total_loss}')
    losses.append(total_loss)

Epoch: 0 Loss: 107236.33376544714
Epoch: 1 Loss: 95285.1626303196
Epoch: 2 Loss: 90054.79716913402
Epoch: 3 Loss: 85579.33580911905
Epoch: 4 Loss: 81242.38136757538
Epoch: 5 Loss: 76810.5693998728
Epoch: 6 Loss: 72152.64379096776
Epoch: 7 Loss: 67188.47575603705
Epoch: 8 Loss: 61902.89307149127
Epoch: 9 Loss: 56291.62704172917
Epoch: 10 Loss: 50409.63848543307
Epoch: 11 Loss: 44312.097784867976
Epoch: 12 Loss: 38077.72647623811
Epoch: 13 Loss: 31784.300539417396
Epoch: 14 Loss: 25522.276401780924
Epoch: 15 Loss: 19465.487217001224
Epoch: 16 Loss: 13928.539531975515
Epoch: 17 Loss: 9336.119049779376
Epoch: 18 Loss: 6003.919225031061
Epoch: 19 Loss: 3889.623540872404
Epoch: 20 Loss: 2654.6825892680454
Epoch: 21 Loss: 1942.525172117291
Epoch: 22 Loss: 1512.0667017639944
Epoch: 23 Loss: 1232.7635919937125
Epoch: 24 Loss: 1039.2191277147354
Epoch: 25 Loss: 897.8849595499289
Epoch: 26 Loss: 790.2070230315828
Epoch: 27 Loss: 705.5808109979341
Epoch: 28 Loss: 637.2554610794936
Epoch: 29 Loss: 